<a href="https://colab.research.google.com/github/harshitadd/CrossPriv/blob/master/PreVID_PySyft_Federated_Averaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install syft==0.2.6 --quiet 
# !pip install pydicom --quiet

In [1]:
import os 
from google.colab import drive
drive.mount('/content/drive')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy
from torch.utils.data import TensorDataset, DataLoader
import time
import csv 
import copy
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from syft.frameworks.torch.fl import utils
from syft.workers.websocket_client import WebsocketClientWorker
import pydicom 
import pandas as pd 
import cv2
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler 
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
def makeNonFedData():
  dcm_path=os.listdir('/content/drive/My Drive/Fed_Covid/minibatch/')
  dcm_data={}
  alpha = 1.5 
  beta = 0
  labels=[]
  pid=[]
  dicom=[]
  label=[]
  for file in dcm_path:
    name = '/content/drive/My Drive/Fed_Covid/minibatch/' + file
    temp = pydicom.dcmread(name)
    image = temp.pixel_array
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    adjusted = cv2.resize(image,(128,128))
    dcm_data[file]=adjusted  
  with open('/content/drive/My Drive/Fed_Covid/stage_2_train_labels.csv','r') as file:
    reader = csv.reader(file)
    for row in reader:
      labels.append(row)  
  scaler = MinMaxScaler()
  cid = 0
  for PID in labels:
    for key in dcm_data:
      if(key[:-4]==PID[0]):
        l=[]
        for val in dcm_data[key]:
          l.append(scaler.fit_transform(val))
        l = np.reshape(l,(3,128,128))
        dicom.append(l)
        label.append(int(PID[5]))
  
  return dicom,label
    

In [3]:
dicom, label = makeNonFedData()

In [4]:
class Arguments():
    def __init__(self):
        self.batch_size = 4
        self.test_batch_size = 4
        self.epochs = 1
args = Arguments()

In [5]:
x_train, x_test, y_train, y_test = train_test_split(dicom,label,test_size=0.3)
test_df = pd.DataFrame()
test_df['features']=x_test
test_df['labels']=y_test
x_train= np.array(x_train)
y_train= np.array(y_train)
x_test= np.array(x_test)
y_test= np.array(y_test)

In [6]:
class data_maker(Dataset):
  def __init__(self, images, labels):
        self.data = images 
        self.targets = labels 

        self.to_torchtensor()
        
  def __len__(self):
      return len(self.data)

  def to_torchtensor(self):      
      self.data=torch.from_numpy(self.data)
      self.labels=torch.from_numpy(self.targets)
  
  def __getitem__(self, idx):
      sample=self.data[idx]
      target=self.targets[idx]
      return sample,target

In [7]:
hook = sy.TorchHook(torch)
hospital = sy.VirtualWorker(hook, id="hospital")  
clinic = sy.VirtualWorker(hook, id="clinic")  

In [8]:
federated_train_data = data_maker(x_train,y_train).federate((hospital,clinic))

In [9]:
federated_train_loader = sy.FederatedDataLoader(federated_train_data,batch_size=args.batch_size)
test_data = data_maker(x_test,y_test)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=args.test_batch_size)

In [31]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()  
        self.conv1 = nn.Conv2d(3,32, kernel_size=8, stride = 2)  
        self.pool1 = nn.MaxPool2d(kernel_size=4,stride=2,padding=0)  
        self.conv2 = nn.Conv2d(32, 64, 8)
        self.pool2 = nn.MaxPool2d(8,8,padding=0)

        # Linear Layers 
        
        self.fc1 = nn.Linear(64*2*2, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        # print(x.shape)
        x = F.relu(self.conv1(x))
        # print(x.shape)
        x = self.pool1(x)
        # print(x.shape)
        x = F.relu(self.conv2(x))
        # print(x.shape)
        x = self.pool2(x)
        # print(x.shape)
        x = x.view(-1,64*2*2)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
 
        return F.log_softmax(x,dim=1)

In [32]:
hospital_model = Net().double()
clinic_model = Net().double()
hospital_optimizer = optim.Adam(hospital_model.parameters(), lr=0.0001)
clinic_optimizer = optim.Adam(clinic_model.parameters(), lr=0.0001)
models = [hospital_model, clinic_model]
optimizers = [hospital_optimizer, clinic_optimizer]
model = Net().double()

In [33]:
compute_nodes = [hospital, clinic]

In [34]:
def update(data, target, model, optimizer):
    loss = nn.CrossEntropyLoss()
    model.send(data.location) 
    optimizer.zero_grad()
    output = model(data)
    out = loss(output, target)
    out.backward()
    optimizer.step()
    return model

In [35]:
def train():
  for batch_idx, (data, target) in enumerate(federated_train_loader):
    for index in range(len(compute_nodes)):
      models[index] = update(data,target,models[index],optimizers[index]) 
    for model in models:
      model.get()
    return utils.federated_avg({"hospital": models[0],"clinic":models[1]}) 


In [47]:
for i,val in enumerate(federated_train_loader):
  print(val[0][0][0][0][0])

(Wrapper)>[PointerTensor | me:49669881127 -> hospital:62090774770]
(Wrapper)>[PointerTensor | me:99650303896 -> hospital:96417058942]
(Wrapper)>[PointerTensor | me:56268577334 -> hospital:4376112198]
(Wrapper)>[PointerTensor | me:58837496549 -> hospital:87139879855]
(Wrapper)>[PointerTensor | me:87132144047 -> hospital:1140471215]
(Wrapper)>[PointerTensor | me:95986700245 -> hospital:68118093444]
(Wrapper)>[PointerTensor | me:82160064218 -> hospital:36666929132]
(Wrapper)>[PointerTensor | me:10447572305 -> hospital:60526882959]
(Wrapper)>[PointerTensor | me:95063713365 -> hospital:82608196782]
(Wrapper)>[PointerTensor | me:98663237637 -> hospital:12532750603]
(Wrapper)>[PointerTensor | me:50081789567 -> hospital:58863684355]
(Wrapper)>[PointerTensor | me:92288627231 -> hospital:62430943806]
(Wrapper)>[PointerTensor | me:36373331765 -> hospital:19609013017]
(Wrapper)>[PointerTensor | me:82397478200 -> hospital:13819709098]
(Wrapper)>[PointerTensor | me:77829321630 -> hospital:7491504226

In [45]:
for i,val in enumerate(test_loader):
  print(val[0][0][0][0][0])

tensor(0.0274, dtype=torch.float64)
tensor(0.0152, dtype=torch.float64)
tensor(0.0934, dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0.7956, dtype=torch.float64)
tensor(1., dtype=torch.float64)
tensor(0.0909, dtype=torch.float64)
tensor(0.9360, dtype=torch.float64)
tensor(0.5075, dtype=torch.float64)
tensor(0.6449, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64)
tensor(0.3404, dtype=torch.float64)
tensor(0.0110, dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0.1429, dtype=torch.float64)
tensor(0.0465, dtype=torch.float64)
tensor(0.2174, dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(1., dtype=torch.float64)
tensor(0.9771, dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0.0133, dtype=torch.float64)
tensor(0.3661, dtype=torch.float64)
tensor(0.5419, dtype=torch.float64)
tensor(0.0156, dtype=torch.float64)
tensor(0.0765, dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor(0.6449, dtype=tor

In [48]:
def test(federated_model):
  federated_model.eval()
  test_loss = 0
  correct = 0
  loss = nn.CrossEntropyLoss()
  for data, target in test_loader:
        output = model(data)
        out = loss(output,target)
        test_loss += out
        #print('Test loss of the iteration on local model - ',test_loss)
        pred = output.argmax(1, keepdim=True)
        # print('predicted val',pred)
        # print('target val',target)
        correct += pred.eq(target.view_as(pred))

  test_loss /= len(test_loader.dataset)
  print(correct)
  print('For Global Model Test loss:' + str(test_loss))

In [37]:
for epoch in range(2):
  start_time = time.time()
  print(f"Epoch Number {epoch}")
  federated_model = train()
  test(federated_model)
  total_time = time.time() - start_time 
  print('Comm time', round(total_time,2),'s\n')

Epoch Number 0
Test loss of the iteration on local model -  tensor(0.6888, dtype=torch.float64, grad_fn=<AddBackward0>)
Test loss of the iteration on local model -  tensor(1.3792, dtype=torch.float64, grad_fn=<AddBackward0>)
Test loss of the iteration on local model -  tensor(2.0667, dtype=torch.float64, grad_fn=<AddBackward0>)
Test loss of the iteration on local model -  tensor(2.7630, dtype=torch.float64, grad_fn=<AddBackward0>)
Test loss of the iteration on local model -  tensor(3.4558, dtype=torch.float64, grad_fn=<AddBackward0>)
Test loss of the iteration on local model -  tensor(4.1445, dtype=torch.float64, grad_fn=<AddBackward0>)
Test loss of the iteration on local model -  tensor(4.8380, dtype=torch.float64, grad_fn=<AddBackward0>)
Test loss of the iteration on local model -  tensor(5.5260, dtype=torch.float64, grad_fn=<AddBackward0>)
Test loss of the iteration on local model -  tensor(6.2152, dtype=torch.float64, grad_fn=<AddBackward0>)
Test loss of the iteration on local mode